In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import os

In [2]:
TEST_DATA_PATH =  "./test"
TRAIN_DATA_PATH = "./train"

In [4]:
# data transform, you can add different transform methods
img_size = 224
torch.manual_seed(0)
train_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

train_aug0 = transforms.Compose([
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])
train_aug1 = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.RandomAffine(0,scale=(0.7,1.2)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

train_aug2 = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.transforms.RandomRotation(25),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])



test_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

dataset_aug0 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug0)
dataset_aug1 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug1)
dataset_aug2 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug2)
dataset_orig = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_transform)

dataset = data.ConcatDataset([dataset_orig, dataset_aug0, dataset_aug1, dataset_aug2])
test_data = datasets.ImageFolder(root=TEST_DATA_PATH,transform=test_transform)

# spilt data into training and validation
TOTAL_SIZE = len(dataset)
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
val_len = round(TOTAL_SIZE * (1-ratio))

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])

# data loader, you can choose the input arguments by yourself

train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=128, shuffle=True,  num_workers=4)
test_data_loader  = data.DataLoader(test_data, batch_size=8, shuffle=False, num_workers=4) 

In [5]:
class ResNet(nn.Module):
    def __init__(self,basicblock,num_classes=5):
    
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1=nn.Sequential(BasicBlock(inplanes=64, planes=64, stride=1),BasicBlock(inplanes=64, planes=64, stride=1))
        self.layer2=nn.Sequential(BasicBlock(inplanes=64, planes=128, stride=2),BasicBlock(inplanes=128, planes=128, stride=1))
        self.layer3=nn.Sequential(BasicBlock(inplanes=128, planes=256, stride=2),BasicBlock(inplanes=256, planes=256, stride=1))
        self.layer4=nn.Sequential(BasicBlock(inplanes=256, planes=512, stride=2),BasicBlock(inplanes=512, planes=512, stride=1))
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512,5)
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet18(pretrained=False, **kwargs):
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model

In [6]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes,kernel_size=3,stride=stride,padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = None
        if(stride==2):
            self.downsample = nn.Sequential(nn.Conv2d(inplanes, planes,kernel_size=1,stride=2),nn.BatchNorm2d(planes))
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [7]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model = ResNet(BasicBlock)
model.to(device=device)

            
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = optim.Adam(params_to_update, lr=0.001)
criterion = nn.CrossEntropyLoss()

# start training 
epochs = 50

min_val_loss = float("inf")

for epoch in range(1, epochs+1):
    model.train()
    total_train_loss = 0
    total_val_loss = 0
    train_hit = 0
    val_hit = 0
    
    for data, target in train_data_loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()

        output=model(data)
        
        # loss function
        loss = criterion(output, target)

        total_train_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_hit += pred.eq(target.data.view_as(pred)).cpu().sum().item() 


        # do back propagation
        loss.backward()
        optimizer.step()
    #lr_sch.step()    
    
    
    with torch.no_grad():
        model.eval()
        for data, target in val_data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_val_loss += F.cross_entropy(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            val_hit += pred.eq(target.data.view_as(pred)).cpu().sum().item() 
    
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_val_loss   = total_val_loss/len(val_data_loader)
    
    print('Epoch:%3d'%epoch
        , '|Train Loss:%8.4f'%(avg_train_loss)
        , '|Train Acc:%3.4f'%(train_hit/len(train_data_loader.dataset)*100.0)
        , '|Val Loss:%8.4f'%(avg_val_loss)
        , '|Val Acc:%3.4f'%(val_hit/len(val_data_loader.dataset)*100.0))
    
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "model.pth")
    

Epoch:  1 |Train Loss:  1.2618 |Train Acc:50.7762 |Val Loss:  1.0455 |Val Acc:55.3191
-------------saving model--------------
Epoch:  2 |Train Loss:  1.0311 |Train Acc:57.8299 |Val Loss:  1.0731 |Val Acc:55.1672
Epoch:  3 |Train Loss:  0.9862 |Train Acc:59.6186 |Val Loss:  1.0144 |Val Acc:60.3343
-------------saving model--------------
Epoch:  4 |Train Loss:  0.9567 |Train Acc:62.0823 |Val Loss:  0.9695 |Val Acc:63.3739
-------------saving model--------------
Epoch:  5 |Train Loss:  0.9586 |Train Acc:60.7830 |Val Loss:  1.1413 |Val Acc:58.0547
Epoch:  6 |Train Loss:  0.9459 |Train Acc:62.7405 |Val Loss:  1.0869 |Val Acc:57.7508
Epoch:  7 |Train Loss:  0.8915 |Train Acc:64.4786 |Val Loss:  0.8319 |Val Acc:67.4772
-------------saving model--------------
Epoch:  8 |Train Loss:  0.8850 |Train Acc:65.2886 |Val Loss:  0.8497 |Val Acc:64.4377
Epoch:  9 |Train Loss:  0.8382 |Train Acc:66.3854 |Val Loss:  0.9893 |Val Acc:65.6535
Epoch: 10 |Train Loss:  0.8305 |Train Acc:67.5329 |Val Loss:  0.80

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model so that you don't need to train the model again
test_model = torch.load("model.pth").to(device)

In [14]:
def test(model,data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        bs = test_data_loader.batch_size
        result = []
        for i, (data, target) in enumerate(test_data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            arr = pred.data.cpu().numpy()
            for j in range(pred.size()[0]):
                file_name = test_data.samples[i*bs+j][0].split('/')[-1]
                result.append((file_name,pred[j].cpu().numpy()[0]))
    return result

In [15]:
result = test(test_model,test_data_loader)

In [16]:
with open ('ID_result.csv','w') as f:
    f.write('ID,label\n')
    for data in result:
        f.write(data[0]+','+str(data[1])+'\n')